In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('logistic_regression_stroke').getOrCreate()
from pyspark.ml.classification import LogisticRegression

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/13 05:40:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("./database/df_con.csv",inferSchema=True,header=True)
data.columns

['_c0',
 'id1',
 'age',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke',
 'id9',
 'age_grade']

In [3]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [4]:
assembler = VectorAssembler(inputCols=[
 'hypertension',
 'heart_disease',
# 'avg_glucose_level',
 'bmi',
# 'smoking_status',
 'age_grade'],outputCol='features')

In [5]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [6]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='stroke',aggregationDepth=2)

In [7]:
pipeline = Pipeline(stages=[assembler,log_reg_titanic])

In [8]:
train_data, test_data = data.randomSplit([0.7,.3])

In [9]:
fit_model = pipeline.fit(train_data)

22/10/13 05:40:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv
22/10/13 05:40:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


In [10]:
results = fit_model.transform(test_data)

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='stroke')

In [12]:
results.select('stroke','prediction').show()

+------+----------+
|stroke|prediction|
+------+----------+
|     1|       1.0|
|     1|       0.0|
|     1|       0.0|
|     1|       1.0|
|     1|       1.0|
|     1|       0.0|
|     1|       1.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       1.0|
|     1|       0.0|
|     1|       0.0|
+------+----------+
only showing top 20 rows



22/10/13 05:40:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


In [13]:
AUC = my_eval.evaluate(results)

AUC

22/10/13 05:40:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


0.5976190476190476

In [14]:
npre    = results.filter(results['stroke']==1).filter(results['prediction']==1.0).count()
nstroke = results.filter(results['stroke']==1).count()
npre/nstroke

22/10/13 05:40:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv
22/10/13 05:40:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


0.25384615384615383